In [2]:
import numpy as np
import numpy.fft as fft
import matplotlib.pyplot as plt
import seaborn as sns
#from scipy.constants import c, pi
sns.set_style('whitegrid')
import radarProc

In [118]:
class Proc:
    
    def __init__(self, Pfa=1e-8, Rmax=50e3):
        self.Rmax = Rmax
        self.Pfa = Pfa
        self.ADC = radarProc.ADC(Rmax=self.Rmax)
        self.flags = np.zeros(4)
        self.delF = 350
        #self.fig, self.ax = self.init_graph()
       
    
    def init_graph(self):
        fig, axes = plt.subplots(3, 2, figsize=(10, 8), subplot_kw=dict(projection='polar'))
        for ax1, ax2 in axes:
            ax1.set_thetamin(45)
            ax1.set_thetamax(135)
            
            ax2.set_thetamin(45)
            ax2.set_thetamax(135)
            
        fig.tight_layout()
        
        return fig, axes
    
    
    def lcm(self, array):
        return np.lcm.reduce(array)
    
        
    def coincidence(self, bins, n_max):
        flags = np.zeros(4)
        n_PRF = len(bins)
        L = [row.shape[1] for row in bins]
        Lu_max = self.lcm(L)
        n_det = [np.count_nonzero(row) for row in bins]
        # Target missed
        flags[0] = not(all(n_det))

        targets = bins
        for i in range(n_PRF):
            # Repeat array
            n_rep = int(np.floor(n_max/L[i])) + 1
            targets[i] = np.tile(bins[i], (1, n_rep))
        
        big = max([row.shape[1] for row in targets])
        for i in range(n_PRF):
            # match array length to concatenate
            e = big-targets[i].shape[1]
            bins[i] = np.pad(targets[i], [(0,0), (0,e)], mode='constant')
        
        match = np.concatenate(targets)
        index = np.where(match.sum(axis=0) == n_PRF)[0]
    
        # Long range ambiguity
        flags[1] = np.logical_and(Lu_max < index, index < n_max).any()
        # Too many targets
        flags[2] = len(index) > max(n_det)
        
        self.flags = flags
      
        return index, flags   
    
    
    def dwell(self, n_P):
        # Take n_P dwells at current PRF
        dwell = np.zeros((n_P, self.ADC.L), dtype='complex128')
        for i in range(n_P):
            measurement = self.ADC.read()
            dwell[i,:] = measurement[0] + 1j*measurement[1]
        
        return dwell
    
            
    def process_dwell(self, dwell):
        dwell = fft.fft(dwell, axis=0)        
        if self.ADC.EL == 1:
            dwell[0, :] = 0
            
        return fft.fftshift(dwell, axes=0)
    
    
    def threshold(self, dwell):
        n_P = dwell.shape[0]
        V_t = np.sqrt(-2*self.ADC.noise_var*n_P*np.log(self.Pfa))
        
        return (np.abs(dwell) > V_t).astype(int)
    
    
    def m_of_n(self, m, n, n_P):
        out = np.zeros((n_P, self.ADC.L, n), dtype='complex128')
        for i in range(n):
            dwell = self.dwell(n_P)
            processed = self.process_dwell(dwell)
            out[:,:, i] = self.threshold(processed)
            
        return (out.sum(axis=2) > m).astype(int)
    
    
    def to_bins(self, data, mode='range'):
        if mode == 'range':
            return data.any(axis=0, keepdims=True)
        else:
            return data.any(axis=1, keepdims=True).T
        
        
    def PRF_sweep(self, PRFs, mode='single', M=2, N=3):
        range_bins = []
        velocity_bins = []
        for PRF in PRFs:
            self.ADC.set_PRF(PRF)
            n_P = int(PRF/self.delF)
            if mode == 'single':
                dwell = self.dwell(n_P)
                dwell = self.process_dwell(dwell)
                thresh = self.threshold(dwell)
                range_bins.append(self.to_bins(thresh))
            else:
                #m_of_n
                thresh = self.m_of_n(M, N, n_P)
                range_bins.append(self.to_bins(thresh))
        
        return range_bins
                
            
    
    

In [159]:
proc = Proc()
proc.ADC.EL = 2
#myADC = proc.ADC()

targ1 = radarProc.Target(5e3, 0, 30, EL=2, CPI=np.inf, SNR=0)
targ2 = radarProc.Target(25e3, 0, 10, EL=2, CPI=np.inf)
targ3 = radarProc.Target(45e3, 0, -274, EL=2, CPI=np.inf)
targ3 = radarProc.Target(45e3, 0, 150, EL=2, CPI=np.inf)
proc.ADC.add_target(targ1)
proc.ADC.add_target(targ2)
proc.ADC.add_target(targ3)
proc.ADC.add_target(targ3)

print(proc.ADC.visible_targets())

[ 5000. 25000. 45000. 45000.]


In [160]:
bins = proc.PRF_sweep([17e3, 15e3], mode='m_of_n')

In [161]:
dist, flags = proc.coincidence(bins, 333)
print(dist*150, flags)

[ 4950 24900 45000] [0. 0. 0. 0.]
